In [1]:

from typing import Optional

import torch
from torch import Tensor


def metis(
    rowptr: Tensor,
    col: Tensor,
    num_partitions: int,
    node_weight: Optional[Tensor] = None,
    edge_weight: Optional[Tensor] = None,
    recursive: bool = False,
) -> Tensor:
    r"""Clusters/partitions a graph into multiple partitions via :obj:`METIS`,
    as motivated by the `"Cluster-GCN: An Efficient Algorithm for Training Deep
    and Large Graph Convolutional Networks"
    <https://arxiv.org/abs/1905.07953>`_ paper.

    Args:
        rowptr (torch.Tensor): Compressed source node indices.
        col (torch.Tensor): Target node indices.
        num_partitions (int): The number of partitions.
        node_weight (torch.Tensor, optional): Optional node weights.
            (default: :obj:`None`)
        edge_weight (torch.Tensor, optional): Optional edge weights.
            (default: :obj:`None`)
        recursive (bool, optional): If set to :obj:`True`, will use multilevel
            recursive bisection instead of multilevel k-way partitioning.
            (default: :obj:`False`)

    Returns:
        torch.Tensor: A vector that assings each node to a partition.
    """
    return torch.ops.pyg.metis(rowptr, col, num_partitions, node_weight,
                               edge_weight, recursive)

In [2]:
import torch
import numpy as np
import scipy.sparse as sp

# 创建一个示例图
edge_index = torch.tensor([[0, 1, 1, 2, 2, 3, 3, 0], 
                           [1, 0, 2, 1, 3, 2, 0, 3]])

# 将边索引转换为 scipy 稀疏矩阵的 COO 格式
row = edge_index[0].numpy()
col = edge_index[1].numpy()
data = np.ones(row.shape[0])
num_nodes = edge_index.max().item() + 1
coo_matrix = sp.coo_matrix((data, (row, col)), shape=(num_nodes, num_nodes))

# 将 COO 矩阵转换为 CSR 矩阵
csr_matrix = coo_matrix.tocsr()
rowptr = torch.from_numpy(csr_matrix.indptr).to(torch.int32)
col = torch.from_numpy(csr_matrix.indices).to(torch.int32)

# 指定分区数量
num_partitions = 2

# 调用 metis 方法进行图划分
partitions = metis(rowptr, col, num_partitions, recursive=True)

print(f"Partition assignments with recursive=True: {partitions}")

# 再次调用 metis 方法，使用不同的划分策略
partitions_non_recursive = metis(rowptr, col, num_partitions, recursive=False)

print(f"Partition assignments with recursive=False: {partitions_non_recursive}")

AttributeError: '_OpNamespace' 'pyg' object has no attribute 'metis'

In [ ]:
import torch
if hasattr(torch.ops.pyg, 'metis'):
    print("METIS function is available in PyG")
else:
    print("METIS function is not available in PyG")

METIS function is not available in PyG
